In [1]:
from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
from pathlib import Path
import xhermes as xh

sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/sdtool_load_test/sdtools"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/transients"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/general_functions"))


from plotting_functions import *
from convergence_functions import * 

from hermes3.case_db import *
from hermes3.casedeck import*
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from hermes3.fluxes import *
from hermes3.selectors import *
# from hermes3.balance1d import *

# plt.style.use('ggplot')
plt.rcParams.update({'font.size': 10})
linewidth = 3
markersize = 15



# plt.style.use('ggplot')
plt.style.use('default')
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.grid'] = True
plt.rcParams.update({'font.size': 16})



%load_ext autoreload
%autoreload 2


In [3]:
ds_1D = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2025-01_STEP_1D-2D_comparison/m4ab-tune_albedo_1D_sep_add_1_increased_density_reduced_density_full_power_corrected_conditions')

/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/hermes-3_venv/lib/python3.11/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


# 1D balance

In [8]:
class Balance1D():
    
    def __init__(
        self, 
        ds, 
        ignore_errors = False, 
        normalised = False, 
        verbose = True, 
        use_sheath_diagnostic = False,
        override_vi = False
        ):
        """
        Prepare a 1D particle and heat balance.
        
        Inputs
        ------
        ds : xarray.Dataset
            Dataset with ALL GUARD CELLS loaded in and no guard replacement
        ignore_errors : bool
            If True, ignore lack of sheath heat flux diagnostic
        normalised : bool
            If True, reproduce normalised quantities in Hermes-3 (incl. normalised mass, charge, etc).
            The dataset you provide MUST be also normalised.
        use_sheath_diagnostic : bool
            If True, use the sheath diagnostic variable for heat fluxes, otherwise use calculated value
        override_vi : bool
            If true, override Vi with Cs. Useful if runnin Hermes-3 with no_flow = True
        verbose : bool
            If True, print warnings and diagnostics
            
        """
        
        self.normalised = normalised
        self.ds = ds
        self.dom = ds.isel(y = slice(2,-2))   # Domain
        self.terms_extracted = False
        self.tallies_extracted = False
        self.sheath_diagnostics = True
        self.use_sheath_diagnostic = use_sheath_diagnostic
        self.override_vi = override_vi
        self.verbose = verbose
        # self.get_properties()  # Get settings etc
        # self.reconstruct_sheath_fluxes()  # Now have .sheath dict with properties
        
        # Check for sheath diagnostic variables
        sheath_flux_candidates = [var for var in ds.data_vars if re.search("S.*\+_sheath", var)]
        if len(sheath_flux_candidates) == 0:
            self.sheath_diagnostics = False
        
        # Check if more than one time slice
        if "t" in ds.dims:
            self.time = True
        else:
            self.time = False

    def get_terms(self):
        """
        Extract individual balance terms based on available diagnostics
        Each term in pbal or hbal is in [s^-1] or [W]. All pressure sources
        have been converted to energy sources and everything has been integrated.
        This may cause confusion because the names of the variables are unchanged.
        """
        ds = self.ds
        dom = self.dom
        
        def integral(data):
            return (data * self.dom["dv"]).sum("y").values

        ### Particle balance
        pbal = {}
        pbalsum = {}
        for param in ["Sd+_src", "Sd_src", "Sd+_feedback", "Sd+_iz", "Sd+_rec", "Sd_target_recycle", "Sd+_sheath"]:
            if param in ds:
                pbal[param] = integral(dom[param])
            else:
                if self.verbose:
                    print(f"Warning! {param} not found, results may be incorrect")
                pbal[param] = np.zeros_like(integral(dom["Ne"]))
       
                
        ### Heat balance
        hbal = {}
        hbalsum = {}
        for param in ["Pd+_src", "Pe_src", "Pd_src", "Rd+_ex", "Rd+_rec", "Rar", "Ed_target_recycle", "Ee_sheath", "Ed+_sheath"]:
            if param in ds:
                hbal[param] = integral(dom[param])*1e-6   # MW

                if param.startswith("P"):
                    newname = "E" + param[1:]
                    hbal[newname] = hbal[param] * 3/2    # Convert from pressure to energy
                    del hbal[param]
                    
                if param == "Rar":
                    hbal[param] = -1 * abs(hbal[param])   # Correct inconsistency in sign convention
            else:
                if self.verbose:
                    print(f"Warning! {param} not found, results may be incorrect")
                hbal[param] = np.zeros_like(integral(dom["Ne"]))
                
        if self.use_sheath_diagnostic is False or self.sheath_diagnostics is False:
            if self.verbose: print("|||WARNING: Overwriting sheath diagnostics with calculated values")
            hbal["Ee_sheath"] = self.sheath["hfe"]
            hbal["Ed+_sheath"] = self.sheath["hfi"]
            pbal["Sd+_sheath"] = self.sheath["pfi"]
        
        self.pbal = pbal
        self.hbal = hbal
        
        if self.sheath_diagnostics is False:
            if self.verbose: print("Sheath diagnostics not available, attempting to reconstruct...")
            self.reconstruct_sheath_fluxes()
            
        self.terms_extracted = True
            

In [9]:
Balance1D(ds_1D)




AttributeError: 'Balance1D' object has no attribute 'reconstruct_sheath_fluxes'